In [8]:
import numpy as np
import seaborn as sns

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from ipywidgets import interact

from flask import Flask, jsonify


In [9]:
# create and save database
execfile("Setup and Organization/Database Setup.py")
# database setup
engine = create_engine("sqlite:///Resources/database.sqlite")
conn=engine.connect()

# reflect an existing database into a new model
base= automap_base()
# reflect the tables
base.prepare(engine,reflect=True)


/var/folders/vd/r058js812ld0xgwzbqh1j9c00000gn/T/ipykernel_52455/525720125.py:10: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  base.prepare(engine,reflect=True)


NameError: name 'database_setup' is not defined